In [38]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [39]:

class OSELM_PT:
    def __init__(self, input_size, hidden_size, activation='sigmoid'):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.activation = activation
        self.device = torch.device("cpu")

        # Initialize input weights and biases randomly
        self.input_weights = torch.randn(hidden_size, input_size, dtype=torch.float32).to(self.device)
        self.biases = torch.randn(hidden_size, dtype=torch.float32).to(self.device)
        self.beta = None
        self.M = None

    def _activation(self, x):
        if self.activation == 'sigmoid':
            return torch.sigmoid(x)
        elif self.activation == 'tanh':
            return torch.tanh(x)
        elif self.activation == 'relu':
            return F.relu(x)
        else:
            raise ValueError("Unsupported activation function")

    def _hidden_layer_output(self, X):
        return self._activation(torch.matmul(X, self.input_weights.T) + self.biases)

    def fit_initial(self, X0, T0):
        H0 = self._hidden_layer_output(X0)
        H0_T = H0.T
        M = torch.linalg.pinv(H0_T @ H0)
        self.M = M
        self.beta = M @ H0_T @ T0

    def update(self, Xn, Tn):
        Hn = self._hidden_layer_output(Xn)
        Hn_T = Hn.T
        I = torch.eye(Hn.shape[0], dtype=torch.float32).to(self.device)

        temp = torch.linalg.inv(I + Hn @ self.M @ Hn_T)
        self.M = self.M - self.M @ Hn_T @ temp @ Hn @ self.M
        self.beta = self.beta + self.M @ Hn_T @ (Tn - Hn @ self.beta)

    def predict(self, X):
        H = self._hidden_layer_output(X)
        return H @ self.beta

In [40]:
# === Load CSV ===
df = pd.read_csv("../Stud_Hansa_Sim_Flight_data.csv")
X = df.iloc[:, :3].values
y = df.iloc[:, 3].values.reshape(-1, 1)

In [41]:
# === Normalize ===
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

In [42]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [43]:

# === Split ===
initial_size = 30
batch_size = 10

X0 = X[:initial_size]
y0 = y[:initial_size]
X_seq = X[initial_size:]
y_seq = y[initial_size:]


In [44]:
# === Train OS-ELM ===
model = OSELM_PT(input_size=3, hidden_size=20, activation='sigmoid')
model.fit_initial(X0, y0)

for i in range(0, len(X_seq), batch_size):
    X_batch = X_seq[i:i+batch_size]
    y_batch = y_seq[i:i+batch_size]
    model.update(X_batch, y_batch)

In [45]:

# === Predict and Evaluate ===
y_pred = model.predict(X).detach().numpy()
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_true_inv = scaler_y.inverse_transform(y.numpy())

mse = mean_squared_error(y_true_inv, y_pred_inv)
r2 = r2_score(y_true_inv, y_pred_inv)

print(f"MSE: {mse:.10f}")
print(f"R² Score: {r2:.10f}")

MSE: 0.0000111596
R² Score: 0.0595120192
